## Add source code folder to PATH

In [ ]:
import os
import sys

sys.path.append(os.path.abspath("../labs"))

## Django setup

In [ ]:
from django.core.wsgi import get_wsgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "True")

os.environ.setdefault("LOCAL_LLM_HOST", "http://localhost:11434")
os.environ.setdefault("DATABASE_HOST", "localhost")

application = get_wsgi_application()

In [ ]:
ISSUE = "ADD YOUR ISSUE TEXT HERE"
EMBEDDER_MODEL_NAME = "nomic-embed-text:latest"
LLM_MODEL_NAME = "llama3.2:latest"
REPO = "."

## Embeddings

In [ ]:
from embeddings.embedder import Embedder
from embeddings.ollama import OllamaEmbedder

embedder = Embedder(OllamaEmbedder, EMBEDDER_MODEL_NAME)

## Vectorize

In [ ]:
from embeddings.vectorizers.chunk_vectorizer import ChunkVectorizer
from embeddings.vectorizers.vectorizer import Vectorizer

Vectorizer(ChunkVectorizer, embedder).vectorize_to_database(None, repository_path=REPO)

## Embeddings:lookup

In [ ]:
embeddings_results = embedder.retrieve_embeddings(ISSUE, REPO)

similar_embeddings = [(embedding.repository, embedding.file_path, embedding.text) for embedding in embeddings_results]

In [ ]:
similar_embeddings

## Prompt

In [ ]:
from tasks.llm import get_context, get_prompt

prompt = get_prompt(ISSUE)
prepared_context = get_context(similar_embeddings, prompt)

## LLM

In [ ]:
from llm.ollama import OllamaRequester
from llm.requester import Requester
from tasks.checks import run_response_checks

requester = Requester(OllamaRequester, model=LLM_MODEL_NAME)
llm_response = requester.completion_without_proxy(prepared_context)
redo, redo_reason = run_response_checks(llm_response)

In [ ]:
from pprint import pprint

pprint(llm_response[1]["choices"][0]["message"]["content"].replace("\\n", " "))